In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/bluebook-for-bulldozers/Train.zip")

In [ ]:
pd.set_option('display.max_columns', None)
df.head(5)

In [ ]:
operations = []

# Feature Selection

In [ ]:
will_drop = []
maybe_will_drop = []

In [ ]:
will_drop.append("MachineID")
will_drop.append("ModelID")
will_drop.append("datasource")

In [ ]:
from pylab import rcParams

rcParams['figure.figsize'] = 18, 8

In [ ]:
df.isna().sum()

### Year Made

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df.YearMade.value_counts()

! We will drop datas that have year value of 1000.

In [ ]:
df = df[df.YearMade > 1000]

In [ ]:
a = list(df.YearMade.unique())
b = [df.SalePrice[df.YearMade == i].sum() / df.SalePrice[df.YearMade == i].count()  for i in a]
plt.bar(a,b)

In [ ]:
def editYearMade(df):
    return df[df.YearMade > 1000]
operations.append(editYearMade)

### MachineHoursCurrentMeter

In [ ]:
df.MachineHoursCurrentMeter.isnull().sum()

In [ ]:
len(list(df.MachineHoursCurrentMeter.unique()))

In [ ]:
df.MachineHoursCurrentMeter = df.MachineHoursCurrentMeter.apply(lambda x : x // 10000)

In [ ]:
a = list(df.MachineHoursCurrentMeter.unique())
b = [df.SalePrice[df.MachineHoursCurrentMeter == i].sum() / df.SalePrice[df.MachineHoursCurrentMeter == i].count()  for i in a]
plt.bar(a,b)

Maybe We must drop this feature

In [ ]:
will_drop.append('MachineHoursCurrentMeter')

### UsageBand

In [ ]:
df.UsageBand.isnull().sum()

In [ ]:
a = ["Low" , "Medium", "High"]
b = []
for i in a:
    b.append(df.SalePrice[df.UsageBand == i].sum() / df.SalePrice[df.UsageBand == i].count())
plt.bar(a,b)

! We have a lot of null valus on this feature but we will try to fill nulls. This feature looks like good to model.

In [ ]:
df[df.UsageBand == "Low"].describe().YearMade

In [ ]:
df[df.UsageBand == "Medium"].describe().YearMade

In [ ]:
df[df.UsageBand == "High"].describe().YearMade

In [ ]:
df[df.UsageBand == "Low"].describe().MachineHoursCurrentMeter

In [ ]:
df[df.UsageBand == "Medium"].describe().MachineHoursCurrentMeter

In [ ]:
df[df.UsageBand == "High"].describe().MachineHoursCurrentMeter

In [ ]:
df[df.UsageBand == "Low"].describe().SalePrice

In [ ]:
df[df.UsageBand == "Medium"].describe().SalePrice

In [ ]:
df[df.UsageBand == "High"].describe().SalePrice

In [ ]:
df.UsageBand.isnull().sum()

In [ ]:
def predictUsageBand(ref, low, medium, high):
    x1 = abs(ref - low)
    x2 = abs(ref - medium)
    x3 = abs(ref - high)
    min_ = min(x1, x2, x3)
    if min_ == x1:
        return "Low"
    elif min_ == x2:
        return "Medium"
    else:
        return "High"

In [ ]:
def predictUsage(hours, usageBand, price, yearMade):
    if not pd.isna(usageBand):
        return usageBand
    if not pd.isna(hours):
        return predictUsageBand(hours, 2, 5, 248)
    if not pd.isna(price):
        return predictUsageBand(price, 29000, 37000, 44000)
    if not pd.isna(yearMade):
        return predictUsageBand(yearMade, 1999, 2001, 2002)
    return None
    

In [ ]:
df.UsageBand = df.apply(lambda x : predictUsage(x['MachineHoursCurrentMeter'], 
                                                x['UsageBand'], x['SalePrice'], x['YearMade']), axis = 1)

In [ ]:
df.UsageBand.isnull().sum()

In [ ]:
a = ["Low" , "Medium", "High"]
b = []
for i in a:
    b.append(df.SalePrice[df.UsageBand == i].sum() / df.SalePrice[df.UsageBand == i].count())
plt.bar(a,b)

In [ ]:
def editUsageBand(df):
    df.UsageBand = df.apply(lambda x : predictUsage(x['MachineHoursCurrentMeter'], 
                                                x['UsageBand'], x['SalePrice'], x['YearMade']), axis = 1)
    return df
operations.append(editUsageBand)

### auctioneerID

In [ ]:
df.auctioneerID.isnull().sum()

In [ ]:
df.auctioneerID.value_counts()

In [ ]:
df.auctioneerID.fillna(1.0, inplace = True)

In [ ]:
def editAuctioneerID(df):
    df.auctioneerID = df.auctioneerID.fillna(1.0)
    return df
operations.append(editAuctioneerID)

In [ ]:
df.auctioneerID.isnull().sum()

In [ ]:
df.auctioneerID.unique()

In [ ]:
a = list(df.auctioneerID.unique())
b = []
for i in a:
    b.append(df.SalePrice[df.auctioneerID == i].sum() / df.SalePrice[df.auctioneerID == i].count())
plt.bar(a,b)

In [ ]:
maybe_will_drop.append('auctioneerID')

### saledate

In [ ]:
df.saledate.isnull().sum()

In [ ]:
df.saledate

In [ ]:
df.saledate = df.saledate.apply(lambda x : int(x.split("/")[-1][:4]) + (int(x.split("/")[0]) * (1/12)))
df.saledate

! We Can extract new feature from saledate and yearMade

In [ ]:
df["Age"] = df.saledate - df.YearMade
df.Age

In [ ]:
df.Age = df.Age.apply(lambda x : x // 0.25)

In [ ]:
a = list(df.Age.unique())
b = []
for i in a:
    b.append(df.SalePrice[df.Age == i].sum() / df.SalePrice[df.Age == i].count())
plt.bar(a,b)

In [ ]:
def editAge(df):
    df.saledate = df.saledate.apply(lambda x : int(x.split("/")[-1][:4]) + (int(x.split("/")[0]) * (1/12)))
    df["Age"] = df.saledate - df.YearMade
    df.Age = df.Age.apply(lambda x : x // 0.25)
    return df
operations.append(editAge)

In [ ]:
will_drop.append("saledate")

### fiBaseModel

In [ ]:
df.fiBaseModel.isnull().sum(), len(df.fiBaseModel.unique())

We will group baseModels by SalePrice feature because we have 1783 group and this number will slow our train algorithm

In [ ]:
base_model = { base:dict() for base in list(df.fiBaseModel.unique()) }

In [ ]:
x = 0
while x != 100000:
    index = (x // 10000) + 1
    for k,v in dict(df[(df.SalePrice >= x) & (df.SalePrice < (x + 10000))].fiBaseModel.value_counts()).items():
        base_model[k][index] = v
    x += 10000
    
# 100K and more
for k,v in dict(df[df.SalePrice >= 100000].fiBaseModel.value_counts()).items():
        base_model[k][11] = v

for k,v in base_model.items():
    try:
        base_model[k] = sorted(base_model[k].items(), key = lambda x : x[1], reverse = True)[0][0]
    except:
        print(k,v)
        break
#base_model

In [ ]:
df.fiBaseModel = df.fiBaseModel.apply(lambda x : base_model[x])
df.fiBaseModel

In [ ]:
def editBaseModel(df):
    df.fiBaseModel = df.fiBaseModel.apply(lambda x : base_model[x])
    return df
operations.append(df)

In [ ]:
df.fiBaseModel.isnull().sum(), len(df.fiBaseModel.unique())

In [ ]:
df.fiSecondaryDesc.isnull().sum(), len(df.fiSecondaryDesc.unique())

In [ ]:
df.fiModelDesc.isnull().sum() ,len(df.fiModelDesc.unique())

In [ ]:
df.fiModelDescriptor.isnull().sum(), len(df.fiModelDescriptor.unique())

In [ ]:
maybe_will_drop.append('fiBaseModel')
will_drop.append('fiModelDesc')
will_drop.append("fiSecondaryDesc")
will_drop.append("fiModelDescriptor")
will_drop.append("fiModelSeries")

### Product Size

In [ ]:
df.ProductSize.isnull().sum(), len(df.ProductSize.unique())

In [ ]:
df.ProductSize.dropna().unique()

In [ ]:
a = list(df.ProductSize.dropna().unique())
b = []
for i in a:
    b.append(df.SalePrice[df.ProductSize == i].sum() / df.SalePrice[df.ProductSize == i].count())
plt.bar(a,b)

As we can see product size is helpful for model. We try to fill null variables

In [ ]:
len(df.fiProductClassDesc.dropna().unique())

In [ ]:
a = ["Medium", "Small", "Large / Medium", "Mini", "Compact", "Large"]
b = []
for i in a:
    b.append( df[df.ProductSize == i].SalePrice.describe().mean() )
plt.bar(a,b)

In [ ]:
a = ["Medium", "Small", "Large / Medium", "Mini", "Compact", "Large"]
product_size_dict = {}
for i in a:
    f = df[df.ProductSize == i].fiProductClassDesc.value_counts()
    for index,count in zip(list(f.index), f):
        if index not in product_size_dict:
            product_size_dict[index] = {}
        if i not in product_size_dict[index]:
            product_size_dict[index][i] = 0
        product_size_dict[index][i] += count
for k,v in product_size_dict.items():
    dict_ = sorted(product_size_dict[k].items(), key = lambda x : x[1], reverse = True)
    product_size_dict[k] = dict_[0][0]
product_size_dict

In [ ]:
a = ["Medium", "Small", "Large / Medium", "Mini", "Compact", "Large"]
product_size_dict_2 = {}
for i in a:
    f = df[df.ProductSize == i].fiBaseModel.value_counts()
    for index,count in zip(list(f.index), f):
        if index not in product_size_dict_2:
            product_size_dict_2[index] = {}
        if i not in product_size_dict_2[index]:
            product_size_dict_2[index][i] = 0
        product_size_dict_2[index][i] += count
for k,v in product_size_dict_2.items():
    dict_ = sorted(product_size_dict_2[k].items(), key = lambda x : x[1], reverse = True)
    product_size_dict_2[k] = dict_[0][0]
product_size_dict_2

In [ ]:
def fillProductSize(productSize,sale_price, base_model, product_class_desc):
    if not pd.isna(productSize):
        return productSize
    if not pd.isna(base_model):
        if base_model in product_size_dict_2:
            return product_size_dict_2[base_model]
    if not pd.isna(product_class_desc):
        if product_class_desc in product_size_dict:
            return product_size_dict[product_class_desc]
    if not pd.isna(sale_price):
        if sale_price > 25000:
            return "Large"
        return "Mini"
    return None

In [ ]:
df.ProductSize = df.apply(lambda x : fillProductSize( x['ProductSize'],
                            x['SalePrice'], x['fiBaseModel'], x['fiProductClassDesc'] ), axis = 1)

In [ ]:
def editProductSize(df):
    df.ProductSize = df.apply(lambda x : fillProductSize( x['ProductSize'],
                            x['SalePrice'], x['fiBaseModel'], x['fiProductClassDesc'] ), axis = 1)
    return df
operations.append(editProductSize)

In [ ]:
df.ProductSize.isnull().sum()

### fiProductClassDesc

In [ ]:
df.fiProductClassDesc.isnull().sum()


In [ ]:
len(list(df.fiProductClassDesc.unique()))

### state

In [ ]:
df.groupby(["state"]).describe().SalePrice

In [ ]:
maybe_will_drop.append("state")

### ProductGroup

In [ ]:
df.ProductGroup.isnull().sum(), len(list(df.ProductGroup.unique()))

In [ ]:
a = list(df.ProductGroup.unique())
b = []
for i in a:
    b.append( df[df.ProductGroup == i].SalePrice.mean()  )
plt.bar(a,b)

### ProductGroupDesc

In [ ]:
df.ProductGroupDesc.isnull().sum(), len(list(df.ProductGroupDesc.unique()))

In [ ]:
a = list(df.ProductGroupDesc.unique())
b = []
for i in a:
    b.append( df[df.ProductGroupDesc == i].SalePrice.mean()  )
plt.bar(a,b)

Product Group and Product Group Desc Same features we will drop that one

In [ ]:
will_drop.append("ProductGroupDesc")

### Drive_System

In [ ]:
df.Drive_System.isnull().sum(), len(list(df.Drive_System.unique()))

In [ ]:
a = list(df.Drive_System.dropna().unique())
b = []
for i in a:
    b.append( df[df.Drive_System == i].SalePrice.mean()  )
plt.bar(a,b)

I think this feature lack of information because of that we will drop this feature

In [ ]:
will_drop.append("Drive_System")

### Enclosure

In [ ]:
def printNullAndUnique(a):
    print(a.isnull().sum(), len(list(a.unique())))

In [ ]:
def graphSalePrice(x):
    a = list(x.dropna().unique())
    b = []
    for i in a:
        b.append( df[x == i].SalePrice.mean()  )
    plt.bar(a,b)

In [ ]:
printNullAndUnique(df.Enclosure)

In [ ]:
graphSalePrice(df.Enclosure)

We will fill null variables

In [ ]:
df.Enclosure.describe()

In [ ]:
df.Enclosure.fillna("OROPS", inplace = True)

In [ ]:
def editEnclosure(df):
    df.Enclosure.fillna("OROPS", inplace = True)
    return df
operations.append(editEnclosure)

In [ ]:
df.Enclosure.isnull().sum()

### Forks

In [ ]:
printNullAndUnique(df.Forks)

In [ ]:
graphSalePrice(df.Forks)

Forks looks helpful for model we can try to fill null variables

In [ ]:
a = ["Yes", "None or Unspecified"]
forks_dict = {}
for i in a:
    f = df[df.Forks == i].fiBaseModel.value_counts()
    for index,count in zip(list(f.index), f):
        if index not in forks_dict:
            forks_dict[index] = {}
        if i not in forks_dict[index]:
            forks_dict[index][i] = 0
        forks_dict[index][i] += count
for k,v in forks_dict.items():
    dict_ = sorted(forks_dict[k].items(), key = lambda x : x[1], reverse = True)
    forks_dict[k] = dict_[0][0]
forks_dict

In [ ]:
def checkForks(forks, base_model):
    if not pd.isna(forks):
        return forks
    if base_model in forks_dict:
        return forks_dict[base_model]
    return "None or Unspecified"
df.Forks = df.apply(lambda x : checkForks(x["Forks"], x["fiBaseModel"]), axis = 1)

In [ ]:
def editForks(df):
    df.Forks = df.apply(lambda x : checkForks(x["Forks"], x["fiBaseModel"]), axis = 1)
    return df
operations.append(editForks)

In [ ]:
printNullAndUnique(df.Forks)

In [ ]:
graphSalePrice(df.Forks)

### Pad_Type

In [ ]:
printNullAndUnique(df.Pad_Type)

In [ ]:
graphSalePrice(df.Pad_Type)

In [ ]:
will_drop.append("Pad_Type")

### Ride_Control

In [ ]:
printNullAndUnique(df.Ride_Control)

In [ ]:
graphSalePrice(df.Ride_Control)

In [ ]:
df.Ride_Control.fillna("None or Unspecified",inplace= True)
graphSalePrice(df.Ride_Control)

In [ ]:
def editRideControl(df):
    df.Ride_Control.fillna("None or Unspecified",inplace= True)
    return df
operations.append(editRideControl)

### Stick

In [ ]:
printNullAndUnique(df.Stick)

In [ ]:
graphSalePrice(df.Stick)

In [ ]:
will_drop.append("Stick")

### Transmission

In [ ]:
printNullAndUnique(df.Transmission)

In [ ]:
graphSalePrice(df.Transmission)

In [ ]:
df.Transmission = df.Transmission.apply(lambda x : 1 if x == "AutoShift" else 0)

In [ ]:
def editTransmission(df):
    df.Transmission = df.Transmission.apply(lambda x : 1 if x == "AutoShift" else 0)
    return df
operations.append(editTransmission)

# Prepare to Train

In [ ]:
df2 = df.iloc[: , :27]
df2["Age"] = df["Age"]

In [ ]:
will_drop.append("SalesID")

In [ ]:
df2.drop(columns = will_drop, inplace= True)
df2

In [ ]:
def editDF(df):
    df2 = df.iloc[: , :27]
    df2.drop(columns = will_drop, inplace= True)
    df2["Age"] = df["Age"]
    return df2
operations.append(df)

In [ ]:
df2.isnull().sum()

YearMade, UsageBand, fiModelDesc, fiBaseModel, ProductSize,

fiProductClassDesc, state, ProductGroup, Enclosure, Forks,

Ride_Control are categorical features. We have to convert with label encoding

In [ ]:
categoricals = ["YearMade", "UsageBand", "fiBaseModel", "ProductSize", "fiProductClassDesc", "state", "ProductGroup",
               "Forks", "Ride_Control", "Enclosure"]
for c in categoricals:
    df2[c] = df2[c].astype("category").cat.codes
df2

In [ ]:
df2 = df2.reset_index().drop(columns="index")
df2

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')


for c in categoricals:
    enc_df = pd.DataFrame(enc.fit_transform(df2[[c]]).toarray())
    df2 = df2.drop(columns = c).join(enc_df, lsuffix = c[0])
df2

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df2.drop(columns = "SalePrice"), df2.SalePrice,train_size = 0.8)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size = 0.5)

# Train and Hyperparameter Tuning

### Choose Subsample Size

First we must choose a subsumple number that resemble our data. Because we have a lot of data we lose time while parameter tuning.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
clf = RandomForestRegressor()

In [ ]:
nums = [1000, 3000, 5000, 10000, 20000]
stds = []
accs = []
for i in nums:
    acc = []
    for k in range(5):
        per = np.random.permutation(X_train.shape[0])[:i]
        clf = RandomForestRegressor()
        clf.fit(X_train.iloc[per], y_train.iloc[per])
        acc.append(r2_score( y_val, clf.predict(X_val) ))
    stds.append(np.std(acc))
    accs.append(np.mean(acc))
stds, accs

In [ ]:
plt.bar(nums, stds, width = 1000)
plt.title("Standard Deviation of Subsamples")

In [ ]:
plt.bar(nums, accs, width = 1000)
plt.ylim(0.7, 1)
plt.title("Accuracy of Subsamples")

We can choose 10000 as subsample size

### Choose Parameters for RF

In [ ]:
per = np.random.permutation(X_train.shape[0])[:10000]
test_x = X_train.iloc[per]
test_y = y_train.iloc[per]
nums = [1, 3, 10, 15, 30, 50, 75, 100]
acc_val = []
acc_test = []
for i in nums:
    clf = RandomForestRegressor(n_estimators=i)
    clf.fit(test_x, test_y)
    acc_val.append(r2_score( y_val, clf.predict(X_val) ))
    acc_test.append( r2_score( y_test, clf.predict(X_test) ) )

In [ ]:
plt.plot(nums, acc_val)
plt.plot(nums, acc_test)

We can choose n_estimator 30, Okay let's look our r2-score on whole dataset.

In [ ]:
%%time
clf = RandomForestRegressor(n_estimators=30)
clf.fit(X_train, y_train)

In [ ]:
print("R2 Validation : ", r2_score(y_val, clf.predict(X_val) ))
print("R2 Test : ", r2_score(y_test, clf.predict(X_test) ))

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
def get_r2_score(clf):
    print("R2 Validation : ", r2_score(y_val, clf.predict(X_val) ))
    print("R2 Test : ", r2_score(y_test, clf.predict(X_test) ))
get_r2_score(clf)

### Neural Network

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(256, input_dim = X_train.shape[1], activation = "relu"))
model.add(keras.layers.Dense(256, activation = "relu"))
model.add(keras.layers.Dense(256, activation = "relu"))
model.add(keras.layers.Dense(1 ,activation = "linear"))
model.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['mean_squared_logarithmic_error'])
model.summary()

In [ ]:
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(
    monitor = "val_mean_squared_logarithmic_error",
    factor = 0.5,
    patience = 3,
    verbose = 1,
)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size = 32,
         callbacks = [learning_rate_reduction])

In [ ]:
get_r2_score(model)

In [ ]:
from sklearn.metrics import mean_squared_log_error
from math import sqrt
def rmsle(clf):
    print("R2 Validation : ", sqrt(mean_squared_log_error(y_val, clf.predict(X_val)) ))
    print("R2 Test : ", sqrt(mean_squared_log_error(y_test, clf.predict(X_test)) ))
rmsle(model)